# **AI Candidate Screening Agent using CrewAI and Amazon Nova Premier**

This agent help you analyze your CV against the job requirements, provide result for the next recruitment step and create interview question.

In [1]:
# Install CrewAI and AWS
!pip install python-dotenv packaging crewai pypdf boto3

In [2]:
import os
# Load environment and save AWS token
from dotenv import load_dotenv
load_dotenv("awskey.txt")
# Get AWS access key, AWS secret key and AWS region
accesskey = os.getenv("AWS_ACCESS_KEY")
secretkey = os.getenv("AWS_SECRET_KEY")
region = os.getenv("AWS_REGION")

In [3]:
# Import CrewAI
from crewai import LLM, Agent, Task, Crew, Process

llm = LLM(
    model="bedrock/us.amazon.nova-premier-v1:0",
    temperature=0.0,
    aws_access_key_id=accesskey,
    aws_secret_access_key=secretkey,
    aws_region_name=region
)

# **Structure of this AI Agent**

**User --> Extract CV --> Compare and Match between Job Requirements and CV --> Result to the next step of the recruitment process --> Create Interview Question --> End**

# **AGENT 1: Compare and Match between Job Requirements and CV**

In [4]:
compareAndMatchAgent = Agent(
    role="Compare and Match Agent",
    goal="Compare and match between job requirements and extracted of curriculum vitae",
    backstory="""
    You are an expert HR recruiter with a keen eye for detail.
    Your primary function is to compare and match a candidate curriculum vitae (CV) against job requirements.
    Your insights are crucial for determining a candidate's suitability for the next stages of the recruitment process.
    """,
    verbose=True,
    allow_delegation=False,
    memory=False,
    llm=llm
)

# **AGENT 2: Result to the next step of the recruitment process**

In [5]:
resultNextStepAgent = Agent(
    role="Result to Next Step of the Recruitment Process Agent",
    goal="Provide result to next step of the recruitment process",
    backstory="""
    You are an expert in HR recruitment processes, capable of assessing candidate suitability and
    determining the next logical step in their application journey.
    """,
    verbose=True,
    allow_delegation=False,
    memory=False,
    llm=llm
)

# **AGENT 3: Create Interview Questions**

In [6]:
createInterviewQuestionAgent = Agent(
    role="Create Interview Questions Agent",
    goal="Create interview questions for interview session of the recruitment process",
    backstory="""
    You are an experienced HR interviewer, skilled at formulating insightful and relevant questions.
    Your objective is to design interview questions that effectively probe a candidate's skills,
    experience, and suitability based on their curriculum vitae.
    """,
    verbose=True,
    allow_delegation=False,
    memory=False,
    llm=llm
)

# **TASK 1: Compare and Match between Job Requirements and CV**

In [7]:
compareAndMatchTask = Task(
    description="""
    You are a virtual human resources expert. You help me COMPARE and MATCH between {cv_data} and requirements like this :
    1. Work across the AI lifecycle: from data preparation and model development to evaluation and deployment.
    2. Fine-tune and integrate LLMs (like OpenAI and Claude) into ERP workflows.
    3. Build smart features such as recommendation engines, forecasting modules, NLP tools, and more.
    Preferred requirements :
    Develop and maintain scalable cloud-based AI solutions across multi-cloud platforms (AWS, GCP, Azure).
    """,
    expected_output="""
    MINIMAL REQUIREMENTS ANALYSIS :

    Requirement 1:

    Requirement n (n is the number of requirement):

    PREFERRED REQUIREMENTS ANALYSIS :

    OVERALL MATCH PERCENTAGE :

    Strengths:

    Potential Gaps:
    """,
    agent=compareAndMatchAgent
)

# **TASK 2: Result to the next step of the recruitment process**

In [8]:
resultNextStepTask = Task(
    description="""
    You are a virtual human resources expert.
    Analyze the candidate comparison and matching results from the previous task.
    Based on this analysis, determine and clearly state the next step in the recruitment process.
    You help me provide value from 0 until 10 to the next step of recruitment process.
    """,
    expected_output="""
    RESULT : ..... of 10
    """,
    agent=resultNextStepAgent
)

# **TASK 3: Create Interview Questions**

In [9]:
createInterviewQuestionTask = Task(
    description="""
    You are a virtual human resources expert. If result are is LESS THAN 7 of 10, don't create interview questions.
    Otherwise, create 3 interview questions about CV PDF file that already extracted.
    """,
    expected_output="""
    QUESTION 1 : .....
    QUESTION 2 : .....
    QUESTION 3 : .....
    """,
    agent=createInterviewQuestionAgent
)

# **Create CrewAI Crew**

In [10]:
candidate_screening_crew = Crew(
    agents=[compareAndMatchAgent, resultNextStepAgent, createInterviewQuestionAgent],
    tasks=[compareAndMatchTask, resultNextStepTask, createInterviewQuestionTask],
    process=Process.sequential,
    verbose=True,
)

# **Execute First Agent - Always Winner PDF file**

In [11]:
# Extract CV PDF file
from pypdf import PdfReader
first_reader = PdfReader("Always Winner CV.pdf")
page = first_reader.pages[0]
first_result = page.extract_text()
print(first_result)

Name  :  Always  Winner  
Phone  Number  :  000888999  
Email  Address  :  alwayswinner@gmail.com 
Work  Experience  :   
AI  Engineer,  Winner,  LLC  |  New  York  City,  NY  |  January  1,  2025  –  Present  
Junior  AI  Engineer,  Nerwin,  LLC  |  New  York  City,  NY  |  January  1,  2024  –  December  31,  2024  
Education  :   
Bachelor  of  Artificial  Intelligence  |  University  of  AI  Everywhere  |  New  York  City,  NY  |  2023  
Technical  Skills  :   
●  Programming  Languages:  Python  (TensorFlow,  PyTorch,  Keras,  scikit-learn,  Hugging  
Face
 
Transformers),
 
Java,
 
C++
 ●  Generative  AI:  GANs  (StyleGAN,  BigGAN),  VAEs,  Diffusion  Models,  Autoregressive  
Models,
 
LLMs
 
(GPT,
 
BERT,
 
LLaMA),
 
Stable
 
Diffusion
 ●  Agentic  AI  ●  Machine  Learning:  Supervised  Learning,  Unsupervised  Learning,  Transfer  Learning,  
Model
 
Optimization,
 
Hyperparameter
 
Tuning
 ●  Cloud  Platforms:  AWS  (Sagemaker,  EC2,  S3),  Azure  (Azure  ML),  Google  Cloud 

In [12]:
result = candidate_screening_crew.kickoff(inputs={'cv_data': first_result})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7b7f8597-5ee9-462c-a5a2-c086b0cb7a25                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Compare and Match Agent                                                                                 │
│                                                                                                                 │
│  Task:                                                                                                          │
│      You are a virtual human resources expert. You help me COMPARE and MATCH between Name  :  Always  Winner    │
│  Phone  Number  :  000888999                                                                                    │
│  Email  Address  :  alwayswinner@gmail.com                                                                      │
│  Work  Experience  :                                                                                            │
│  AI  Engineer,  Winner,  LLC  |  New  York  City,  NY  |  January  1,  2025  –  Present                         │
│  Junior  AI  Engineer,  Nerwin,  LLC  |  New  York  City,  NY  |  January  1,  2024  –  December  31,  2024     │
│  Education  :                                                                                                   │
│  Bachelor  of  Artificial  Intelligence  |  University  of  AI  Everywhere  |  New  York  City,  NY  |  2023    │
│  Technical  Skills  :                                                                                           │
│  ●  Programming  Languages:  Python  (TensorFlow,  PyTorch,  Keras,  scikit-learn,  Hugging                     │
│  Face                                                                                                           │
│                                                                                                                 │
│  Transformers),                                                                                                 │
│                                                                                                                 │
│  Java,                                                                                                          │
│                                                                                                                 │
│  C++                                                                                                            │
│   ●  Generative  AI:  GANs  (StyleGAN,  BigGAN),  VAEs,  Diffusion  Models,  Autoregressive                     │
│  Models,                                                                                                        │
│                                                                                                                 │
│  LLMs                                                                                                           │
│                                                                                                                 │
│  (GPT,                                                                                                          │
│                                                                                                                 │
│  BERT,                                                                                                          │
│                                                                                                                 │
│  LLaMA),                                                                                                        │
│                                                                                                                 │
│  Stable                                                                                                         │
│                                                        

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Compare and Match Agent                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  MINIMAL REQUIREMENTS ANALYSIS :                                                                                │
│                                                                                                                 │
│  Requirement 1: Always Winner has experience as an AI Engineer and Junior AI Engineer, which implies working    │
│  across the AI lifecycle, including data preparation, model development, evaluation, and deployment.            │
│                                                                                                                 │
│  Requirement 2: The candidate lists technical skills in LLMs (e.g., GPT, BERT), which are similar to models     │
│  like OpenAI and Claude. This indicates the ability to fine-tune and integrate such models.                     │
│                                                                                                                 │
│  Requirement 3: Their background in machine learning, NLP, and generative AI (e.g., recommendation engines,     │
│  NLP tools) shows they can build required features.                                                             │
│                                                                                                                 │
│  PREFERRED REQUIREMENTS ANALYSIS :                                                                              │
│  The candidate has experience with AWS (Sagemaker, EC2, S3), Azure (Azure ML), and Google Cloud Platform        │
│  (Vertex AI), demonstrating the ability to develop scalable cloud-based AI solutions across multiple            │
│  platforms.                                                                                                     │
│                                                                                                                 │
│  OVERALL MATCH PERCENTAGE : 95%                                                                                 │
│                                                                                                                 │
│  Strengths:                                                                                                     │
│  - Strong technical foundation in AI, machine learning, and generative models.                                  │
│  - Relevant work experience in AI roles.                                                                        │
│  - Proficiency with required programming languages and tools.                                                   │
│  - Experience with all specified cloud platforms.                                                               │
│                                                                                                                 │
│  Potential Gaps:                                                                                                │
│  - No explicit mention of ERP systems experience, though API development skills may cover integration tasks.    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Create Interview Questions Agent                                                                        │
│                                                                                                                 │
│  Task:                                                                                                          │
│      You are a virtual human resources expert. If result are is LESS THAN 7 of 10, don't create interview       │
│  questions.                                                                                                     │
│      Otherwise, create 3 interview questions about CV PDF file that already extracted.                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Create Interview Questions Agent                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  QUESTION 1: Can you describe a project where you utilized Large Language Models (LLMs) such as GPT or BERT?    │
│  Specifically, how did you approach the fine-tuning process, and what were the key challenges you faced during  │
│  model integration?                                                                                             │
│                                                                                                                 │
│  QUESTION 2: Given your experience with AWS, Azure, and Google Cloud Platform, could you provide an example of  │
│  a scalable AI solution you developed? What specific services (e.g., Sagemaker, Azure ML, Vertex AI) did you    │
│  use, and how did you ensure the solution was efficient and cost-effective?                                     │
│                                                                                                                 │
│  QUESTION 3: While your CV highlights your proficiency in machine learning and NLP, could you elaborate on a    │
│  time when you had to build a generative AI feature, such as a recommendation engine? What methodologies did    │
│  you employ, and how did you measure the success of the implemented feature?                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [13]:
print(result)

QUESTION 1: Can you describe a project where you utilized Large Language Models (LLMs) such as GPT or BERT? Specifically, how did you approach the fine-tuning process, and what were the key challenges you faced during model integration?

QUESTION 2: Given your experience with AWS, Azure, and Google Cloud Platform, could you provide an example of a scalable AI solution you developed? What specific services (e.g., Sagemaker, Azure ML, Vertex AI) did you use, and how did you ensure the solution was efficient and cost-effective?

QUESTION 3: While your CV highlights your proficiency in machine learning and NLP, could you elaborate on a time when you had to build a generative AI feature, such as a recommendation engine? What methodologies did you employ, and how did you measure the success of the implemented feature?


╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Crew: crew
├── 📋 Task: 861b6bdd-b3b4-4093-899a-8bc2b2f2eb98
│   Assigned to: Compare and Match Agent
│   Status: ✅ Completed
├── 📋 Task: dbbf79eb-efa0-4bc8-a9bf-d820902e56c2
│   Assigned to: Result to Next Step of the Recruitment Process Agent
│   Status: ✅ Completed
└── 📋 Task: a58b1690-f2f2-4b62-a450-da4ef95ae2a6
    Assigned to: Create Interview Questions Agent
    Status: ✅ Completed

# **Execute Second Agent - Sonny Wawwak PDF file**

In [14]:
# Extract CV PDF file
from pypdf import PdfReader
second_reader = PdfReader("Sonny Wawwak CV.pdf")
page = second_reader.pages[0]
second_result = page.extract_text()
print(second_result)

Sonny  Wawwak  
UI/UX
 
Designer
 
Address
 
:
 
Konoha
 
Phone
 
:
 
+62
 
812-3456-7890
 
Email
 
Address
 
:
 
sonny.wawwak@email.com
 
Skills  ●  UI  Design  (Web  &  Mobile)  ●  UX  Research  &  Analysis  ●  Wireframing  &  Prototyping  ●  Design  Systems  &  Style  Guides  ●  User  Flows  &  Journey  Mapping  ●  Usability  Testing  ●  Tools:  Figma,  Adobe  XD,  Sketch,  InVision,  Photoshop,  Illustrator  ●  HTML,  CSS  (basic  understanding)  ●  Agile  &  Scrum  Workflow  
Professional  Experience  UI/UX  Designer  
PT
 
I
 
Don’t
 
Know
 
–
 
Konoha
 
Jan
 
2022
 
–
 
Present
  Junior  UI/UX  Designer  
Know
 
Money
 
Only
 
Studio
 
–
 
Konoha
 
Aug
 
2020
 
–
 
Dec
 
2021
 
Education  
Bachelor’s  Degree  in  Visual  Communication  Design  
Design
 
Institute
 
-
 
Konoha
 
2016
 
–
 
2020
 


In [15]:
result = candidate_screening_crew.kickoff(inputs={'cv_data': second_result})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7b7f8597-5ee9-462c-a5a2-c086b0cb7a25                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Compare and Match Agent                                                                                 │
│                                                                                                                 │
│  Task:                                                                                                          │
│      You are a virtual human resources expert. You help me COMPARE and MATCH between Sonny  Wawwak              │
│  UI/UX                                                                                                          │
│                                                                                                                 │
│  Designer                                                                                                       │
│                                                                                                                 │
│  Address                                                                                                        │
│                                                                                                                 │
│  :                                                                                                              │
│                                                                                                                 │
│  Konoha                                                                                                         │
│                                                                                                                 │
│  Phone                                                                                                          │
│                                                                                                                 │
│  :                                                                                                              │
│                                                                                                                 │
│  +62                                                                                                            │
│                                                                                                                 │
│  812-3456-7890                                                                                                  │
│                                                                                                                 │
│  Email                                                                                                          │
│                                                                                                                 │
│  Address                                                                                                        │
│                                                                                                                 │
│  :                                                                                                              │
│                                                                                                                 │
│  sonny.wawwak@email.com                                                                                         │
│                                                                                                                 │
│  Skills  ●  UI  Design  (Web  &  Mobile)  ●  UX  Research  &  Analysis  ●  Wireframing  &  Prototyping  ●       │
│  Design  Systems  &  Style  Guides  ●  User  Flows  &  Journey  Mapping  ●  Usability  Testing  ●  Tools:       │
│  Figma,  Adobe  XD,  Sketch,  InVision,  Photoshop,  Il

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Compare and Match Agent                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  MINIMAL REQUIREMENTS ANALYSIS :                                                                                │
│                                                                                                                 │
│  Requirement 1: Sonny Wawwak's CV does not indicate experience with the AI lifecycle, including data            │
│  preparation, model development, or deployment.                                                                 │
│                                                                                                                 │
│  Requirement 2: There is no evidence of Sonny having experience with fine-tuning or integrating LLMs into ERP   │
│  workflows.                                                                                                     │
│                                                                                                                 │
│  Requirement 3: Sonny's background does not include building AI features such as recommendation engines,        │
│  forecasting modules, or NLP tools.                                                                             │
│                                                                                                                 │
│  PREFERRED REQUIREMENTS ANALYSIS :                                                                              │
│                                                                                                                 │
│  Develop and maintain scalable cloud-based AI solutions: Sonny's CV does not mention experience with cloud      │
│  platforms (AWS, GCP, Azure) or developing AI solutions.                                                        │
│                                                                                                                 │
│  OVERALL MATCH PERCENTAGE : 0%                                                                                  │
│                                                                                                                 │
│  Strengths:                                                                                                     │
│  - Strong background in UI/UX design, including tools like Figma, Adobe XD, and Sketch.                         │
│  - Experience with user research, wireframing, prototyping, and usability testing.                              │
│                                                                                                                 │
│  Potential Gaps:                                                                                                │
│  - Lacks experience in AI, machine learning, or data science.                                                   │
│  - No experience with LLMs, ERP workflows, or cloud platforms.                                                  │
│  - No background in developing AI features or scalable solutions.                                               │
│                                                                                                                 │
│  The candidate's expertise is in UI/UX design, which does not align with the technical AI and development       │
│  skills required for the job.                                                                                   │
│                                                        

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Result to Next Step of the Recruitment Process Agent                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  RESULT: 0 of 10                                                                                                │
│                                                                                                                 │
│  Based on the analysis of Sonny Wawwak's CV and the requirements for the position, it is clear that Sonny does  │
│  not meet the necessary qualifications for this role. His expertise lies in UI/UX design, which does not align  │
│  with the technical AI and development skills required. Given the 0% overall match percentage and the           │
│  significant gaps in experience with AI, machine learning, LLMs, ERP workflows, and cloud platforms, the next   │
│  step in the recruitment process is to:                                                                         │
│                                                                                                                 │
│  1. **Reject the Application**: Inform Sonny that he does not meet the current job requirements.                │
│  2. **Provide Feedback**: Offer constructive feedback highlighting his strengths in UI/UX design and the        │
│  specific areas where he lacks the required experience.                                                         │
│  3. **Future Consideration**: Suggest that his profile will be kept on file for any future UI/UX design         │
│  opportunities that may arise.                                                                                  │
│                                                                                                                 │
│  This decision ensures that we maintain high standards for the role while also respecting Sonny's professional  │
│  background and potential fit for other positions.                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Create Interview Questions Agent                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Based on the provided CV and the job requirements, Sonny Wawwak does not meet the necessary qualifications     │
│  for this role. His expertise in UI/UX design does not align with the technical AI and development skills       │
│  required. Therefore, no interview questions will be created at this time. We recommend rejecting the           │
│  application and providing constructive feedback on his strengths and areas for improvement. His profile can    │
│  be kept on file for future UI/UX design opportunities.                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [16]:
print(result)

Based on the provided CV and the job requirements, Sonny Wawwak does not meet the necessary qualifications for this role. His expertise in UI/UX design does not align with the technical AI and development skills required. Therefore, no interview questions will be created at this time. We recommend rejecting the application and providing constructive feedback on his strengths and areas for improvement. His profile can be kept on file for future UI/UX design opportunities.


╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Crew: crew
├── 📋 Task: 861b6bdd-b3b4-4093-899a-8bc2b2f2eb98
│   Assigned to: Compare and Match Agent
│   Status: ✅ Completed
├── 📋 Task: dbbf79eb-efa0-4bc8-a9bf-d820902e56c2
│   Assigned to: Result to Next Step of the Recruitment Process Agent
│   Status: ✅ Completed
└── 📋 Task: a58b1690-f2f2-4b62-a450-da4ef95ae2a6
    Assigned to: Create Interview Questions Agent
    Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a58b1690-f2f2-4b62-a450-da4ef95ae2a6                                                                     │
│  Agent: Create Interview Questions Agent                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯